In [1]:
from io import StringIO
from pdfminer.high_level import extract_text_to_fp
from pdfminer.layout import LAParams
import re

In [2]:
output_string = StringIO()
with open('receipts/example3.pdf', 'rb') as fin:
    extract_text_to_fp(fin, output_string, laparams=LAParams(),
                       output_type='text', codec='None')
# with open('example.pdf', 'rb') as fin:
#     extract_text_to_fp(fin, output_string)
pages = output_string.getvalue().split("")
for i,p in enumerate(pages):
    pages[i] = p.split("\n\n")
    # print(type(pages[i]),pages[i])
    for j,g in enumerate(pages[i]):
        # print(j,g)
        pages[i][j] = g.split("\n")
    # print(f'Page {i}')
    # print(p)

In [7]:
defaultCustomer = "John Smith"
defaultAddr     = "123 Example Ln\nApt A\nSan Francisco, CA 94117\nUnited States of America"

"""
Buyer
    Name
    Account
Shipping
    Ship to
    Ship by
    Ship method
    Tracking Number
Order
    Order Num
    Order Date
Units
    Black
    White
Transaction
    Item total
    Tax
    Shipping Total
    Order Total*
"""

class Order:
    def __init__(self,page) -> None:

        self.buyer = {
            "name": "DEFAULT",
            "account": "DEFAULT",
            "loyal": False
        }
        self.shipping = {
            "to": "DEFAULT",
            "by": "DEFAULT",
            "method": "DEFAULT",
            "tracking": "DEFAULT"
        }
        self.order = {
            "num": "DEFAULT",
            "date": "DEFAULT"
        }
        self.units = {}
        self.transaction = {
            "items": -1,
            "tax": -1,
            "shipping": -1,
            "currency": "$"
        }

        for l in page:
            if l[0] == "Buyer":
                if len(l[1:]) < 2:
                    self.buyer["name"] = None
                    self.buyer["account"] = l[1]
                else:
                    self.buyer["name"] = l[1]
                    self.buyer["account"] = re.sub(r'[\(\)]','',l[2])
                if self.buyer["name"][-2:] == " ★":
                    self.buyer["name"] = self.buyer["name"][:-2]
                    self.buyer["loyal"] = True
            elif l[0] == "Deliver to":
                self.shipping["to"] = "\n".join(l[1:])

    def __str__(self) -> str:
        return str(self.buyer) + "\n" + str(self.order) + "\n" + str(self.units) + "\n" + str(self.transaction) + "\n" + str(self.shipping)

In [8]:
a = Order(pages[1])
print(a)
# a.buyer["name"][-2:]

{'name': 'Abner Benjamin', 'account': 'lxfzip3x', 'loyal': True}
{'num': 'DEFAULT', 'date': 'DEFAULT'}
{}
{'items': -1, 'tax': -1, 'shipping': -1, 'currency': '$'}
{'to': 'DEFAULT', 'by': 'DEFAULT', 'method': 'DEFAULT', 'tracking': 'DEFAULT'}


In [11]:
orders = []
for order in pages:
    for section in order:
        print(section)

['1 item']
['Better Shoulder Buttons for Miyoo Mini']
['1 x $20.00']
['Color: Black']
['Item total']
['Delivery total']
['Order total']
['$20.00']
['$18.00']
['$38.00']
['GetBetterButtons', 'getbetterbuttons.etsy.com']
['Deliver to', 'Joshua Jones', '?? ??? ??? 739-1', '?????? 206?', '???', 'Gyeonggi-do', '16037', 'South Korea']
['Scheduled to dispatch by', 'Sep 9, 2022']
['Order', '#2590589140']
['Order date', 'Sep 6, 2022']
['Buyer', 'YoshiieBear']
['Payment method', 'Paid via Etsy Payments']
['Delivery method', 'Standard International']
['Tracking', '420917169400109206094533184611', 'via Global Postal Shipping']
['Love what you bought?', 'Add a photo and review with the Etsy app.', 'Visit etsy.com/mobile to download.']
['Carbon-Offset Delivery', 'Etsy offsets 100% of the carbon emissions', 'from your delivery. Learn how at', 'etsy.com/impact.']
['']
['1 item']
['Better Shoulder Buttons for Miyoo Mini']
['1 x $20.00']
['Color: White']
['Item total']
['Tax']
['Shipping total']
['Order